<a href="https://colab.research.google.com/github/nntu079/multiclassification/blob/main/multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch import nn
from torch.optim import Adam
import numpy as np
from tqdm import tqdm
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

In [ ]:
texts = [
    "Sản phẩm tốt , đẹp keng , tiếc là ko lên đc android 9 vì ko có sim softbank , chủ shop nhiệt tình.",
     "Nhận máy mình rất bất ngờ máy đẹp lắm ạ\nMới dùng thấy rất tốt về sau thì chưa biết\nShop trả lời rất nhiệt tình\nCảm ơn shop",
     "Máy đẹp quá, hàng đúng chuẩn luôn ,shop rất uy tín giao hàng nhanh chất lượng đúng như quảng bá, giá cả hợp lý, máy mới đẹp, các ứng dụng chạy ổn định, các chức năng cơ bản hoạt động rất tốt, màn hình cảm ứng nhạy, sáng, màu sắc rực rỡ, mình rất hài lòng vừa ý, sẽ ủng hộ shop thêm ở đơn hàng sao,,,",
     "Shop rất uy tín. Máy bị lỗi khay sim, shop đã đổi lại máy khác.",
     "Trừ 1 sao vì báo số liệu pin chưa chính xác nha. Shop tư vấn khá nhiệt tình",
    "Máy tầm 90% về ngoại hình, áp ok ! Nói chung máy vẫn còn đẹp! Thử dùng vài ngày thì mọi thứ tạm ổn! Tặng shop 5 sao",
     ]

labels1 = {
    0:"mẫu mã",
    1:"dịch vụ",
    2:"hiệu năng",
    3:"cấu hình",
    4:"giá",
    5:"other"
}

labels = [
[1,1,0,0,0,0],
[1,1,0,0,0,0],
[1,0,1,1,1,0],
[0,1,0,0,0,1],
[0,1,0,0,0,1],
[1,0,0,0,0,0],
]


In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self,labels, tokenizer, texts):

        self.labels = labels
        self.texts = [tokenizer(text,
                                padding='max_length', max_length=128, truncation=True,
                                return_tensors="pt") for text in texts]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]['input_ids']

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class Classifier(nn.Module):

    def __init__(self,num_class=10):
        super().__init__()
        self.linear1 = nn.Linear(128, 300)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(300, num_class)
        self.softmax = nn.Hardsigmoid()

    def forward(self, input_id):
        input_id = input_id.resize_(1, 128)
        input_id = input_id.to(torch.float32)

        linear1_output = self.linear1(input_id)
        sigmoid = self.sigmoid(linear1_output)
        linear2_output = self.linear2(sigmoid)

        return self.softmax(linear2_output)

In [ ]:
dataset =Dataset(labels=labels,tokenizer=tokenizer,texts=texts)

model = Classifier(num_class=6)

In [ ]:
dataset.get_batch_texts(0)

In [ ]:
model.forward(dataset.get_batch_texts(0))

In [ ]:
def train(model, train_data,learning_rate=0.001, epochs=10):


    train_dataloader = torch.utils.data.DataLoader(
        train_data, batch_size=1, shuffle=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    for _ in range(epochs):

        model.train()
        for train_input, train_label in tqdm(train_dataloader):

            input_id = train_input
            output = model(input_id)

            print(output)
            print(train_label)
            
            batch_loss = criterion(output, train_label.float())
   
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

In [ ]:
train(model,dataset)

In [ ]:
result = model.forward(dataset.get_batch_texts(5)).tolist()[0]

In [ ]:
result

[1.0, 0.8027536869049072, 0.0, 0.0, 0.0, 0.4136916697025299]

In [ ]:
[ 1 if x >= 0.5 else 0 for x in result ]

[1, 1, 0, 0, 0, 0]

In [ ]:
t = [ 1 if x >= 0.5 else 0 for x in result ]

for index,value in enumerate(t):
  if(value == 1):
    print(labels1[index])

mẫu mã
dịch vụ
